# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import panel as pn

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,boguchany,58.3814,97.4531,-0.72,90,100,4.46,RU,1731298656
1,1,waitangi,-43.9535,-176.5597,15.46,88,98,2.68,NZ,1731298431
2,2,bethel,41.3712,-73.4140,12.47,88,100,1.34,US,1731298374
3,3,new norfolk,-42.7826,147.0587,19.84,18,1,0.80,AU,1731298659
4,4,albany,42.6001,-73.9662,8.25,72,100,0.45,US,1731298107


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [18]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    tiles="EsriOceanBase",
    frame_width=650,
    frame_height=500,
    size='Humidity',
    color="City",
    alpha=0.5,
    scale=0.5,
    hover_cols=['City', 'State'])
                   
# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [23]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_weather_df = city_data_df.loc[(city_data_df["Max Temp"]<=30) & 
                                    (city_data_df["Max Temp"]>=20) &
                                    (city_data_df["Cloudiness"]<=90) &
                                    (city_data_df["Country"]!="JP") &
                                    (city_data_df["Country"]!="ID") &
                                    (city_data_df["Country"]!="MX") &
                                    (city_data_df["Country"]!="BR") &
                                    (city_data_df["Country"]!="AR") &
                                    (city_data_df["Humidity"]< 75),:]

print(len(ideal_weather_df))

# Drop any rows with null values
ideal_weather_df = ideal_weather_df.dropna()

# Display sample data
ideal_weather_df

72


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
19,19,nadi,-17.8000,177.4167,29.00,65,40,7.20,FJ,1731298678
47,47,avarua,-21.2078,-159.7750,24.03,69,75,4.63,CK,1731298515
48,48,nova sintra,14.8667,-24.7167,26.94,70,2,6.91,CV,1731298716
55,55,carnarvon,-24.8667,113.6333,24.24,52,48,8.33,AU,1731298445
66,66,dazhong,33.1997,120.4578,20.05,52,0,3.21,CN,1731298739
...,...,...,...,...,...,...,...,...,...,...
544,544,dilling,12.0500,29.6500,21.68,28,58,3.94,SD,1731299340
547,547,angaur state,6.9060,134.1300,28.78,67,26,0.15,PW,1731299343
548,548,mogok,22.9167,96.5000,20.95,64,78,1.12,MM,1731299344
549,549,gizo,-8.1030,156.8419,29.04,70,85,1.76,SB,1731299346


### Step 3: Create a new DataFrame called `hotel_df`.

In [25]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather_df[["City", 
                             "Country", 
                             "Lat", 
                             "Lng", 
                             "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"]=''

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
19,nadi,FJ,-17.8000,177.4167,65,
47,avarua,CK,-21.2078,-159.7750,69,
48,nova sintra,CV,14.8667,-24.7167,70,
55,carnarvon,AU,-24.8667,113.6333,52,
66,dazhong,CN,33.1997,120.4578,52,
...,...,...,...,...,...,...
544,dilling,SD,12.0500,29.6500,28,
547,angaur state,PW,6.9060,134.1300,67,
548,mogok,MM,22.9167,96.5000,64,
549,gizo,SB,-8.1030,156.8419,70,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [26]:
# Set parameters to search for a hotel
radius = 10000
params = {"categories":"accommodation.hotel", 
         "filter":'',
         "bias":'', 
         "apiKey":geoapify_key,
         "limit":1}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lon = row["Lng"]
    lat = row["Lat"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f'circle:{lon},{lat},{radius}'
    params["bias"] = f'proximity:{lon},{lat}'

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
nadi - nearest hotel: President Hotel
avarua - nearest hotel: Paradise Inn
nova sintra - nearest hotel: Residência Ka Dencho
carnarvon - nearest hotel: No hotel found
dazhong - nearest hotel: No hotel found
howrah - nearest hotel: Sun India Guest House
west island - nearest hotel: Cocos Village Bungalows
taoudenni - nearest hotel: No hotel found
bandar-e lengeh - nearest hotel: Old owner do not like foreigners hotel
muzaffarpur - nearest hotel: Hotel Embassy International
praia - nearest hotel: Casa Sodadi
kasungu - nearest hotel: Kasungu Inn
haiku-pauwela - nearest hotel: Inn At Mama's Fish House
tazacorte - nearest hotel: App Leyma
vallenar - nearest hotel: Hotel del Marqués
bubaque - nearest hotel: Chez Julio
mahibadhoo - nearest hotel: No hotel found
noumea - nearest hotel: Hotel Gondwana
geraldton - nearest hotel: Ocean Centre Hotel
riyadh - nearest hotel: White Palace Hotel
gantang - nearest hotel: No hotel found
tolanaro - nearest hotel: Hôtel Mahavokey
alo

,City,Country,Lat,Lng,Humidity,Hotel Name
19,nadi,FJ,-17.8000,177.4167,65,President Hotel
47,avarua,CK,-21.2078,-159.7750,69,Paradise Inn
48,nova sintra,CV,14.8667,-24.7167,70,Residência Ka Dencho
55,carnarvon,AU,-24.8667,113.6333,52,No hotel found
66,dazhong,CN,33.1997,120.4578,52,No hotel found
...,...,...,...,...,...,...
544,dilling,SD,12.0500,29.6500,28,No hotel found
547,angaur state,PW,6.9060,134.1300,67,No hotel found
548,mogok,MM,22.9167,96.5000,64,Mogok Hill
549,gizo,SB,-8.1030,156.8419,70,Gizo Hotel


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [29]:
%%capture --no-display

# Configure the map plot
second_map_plot = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    tiles="EsriOceanBase",
    frame_width=650,
    frame_height=500,
    size='Humidity',
    color="City",
    alpha=0.5,
    scale=0.5,
    hover_cols=['City', 'Country', 'Hotel Name'])
                   
# Display the map
second_map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country,Hotel Name)